In [322]:
import numpy as np
import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.tree import DecisionTreeRegressor

import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator , TransformerMixin
from sklearn import preprocessing
from sklearn import utils
from sklearn.linear_model import LinearRegression
import joblib
from sklearn.model_selection import GridSearchCV 

In [376]:
x_train = pd.read_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\ScaledTrainSet_data.csv' , index_col  = 'index')
x_train

,year,is_GAC,is_maserati,is_daihatsu,is_porsche,is_ds,is_smart,is_seres,is_cadillac,is_דודג',...,is_leasing,is_company,is_private,next_test_in,annual_licensing_fee,car_length,large,small,mini,medium
index,,,,,,,,,,,,,,,,,,,,,
1608,0.952381,0,0,0,0,0,0,0,0,0,...,0,0,1,0.000000,0.645547,0.764326,1,0,0,0
1329,0.761905,0,0,0,0,0,0,0,0,0,...,0,0,1,0.166667,0.188003,0.600759,0,0,0,1
1782,0.333333,0,0,0,0,0,0,0,0,0,...,0,0,1,0.166667,0.050377,0.652751,0,0,0,1
1193,0.523810,0,0,0,0,0,0,0,0,0,...,1,0,0,0.583333,0.027266,0.493359,0,1,0,0
1166,1.000000,0,0,0,0,0,0,0,0,0,...,0,0,1,0.833333,1.000000,0.489943,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1650,0.809524,0,0,0,0,0,0,0,0,0,...,0,0,1,0.333333,0.122825,0.518027,0,1,0,0
1100,0.428571,0,0,0,0,0,0,0,0,0,...,0,0,1,0.416667,0.142041,0.813283,1,0,0,0
1135,0.714286,0,0,0,0,0,0,0,0,0,...,0,0,1,0.000000,0.295767,0.675522,0,0,0,1


# Cross-validation :

first we return the demi-model (the average) and try to be better than him.... 

In [378]:
 # בדיקת מודל אפס: 

def targetTransform(len_target):
    target_ = np.zeros(len_target)
    for i in range (len_target):
        target_[i] +=  x_target.mean()
    return target_

x_train1 , x_test1 , y_train1 , y_test1= sklearn.model_selection.train_test_split(x_train, x_target , test_size=0.1 , random_state = 2) 

pred = targetTransform(len(y_test1))
#print (f"\n{pred}")
print (f"RMSE: {np.sqrt(mean_squared_error(y_test1 , pred))}")

#במידה והמודל היה מחזיר תמיד את ממוצע מחיר הרכבים בטבלה זו הייתה מידת הדיוק שלו
# מדוייק יותר- כרגע הגענו למרחק הממוצע מחיזוי נכון בשקלים.


RMSE: 77634.8801434407


In [380]:
#Dummy-model method:
    #Checking the prices mean, and then its percent.
    #Count the prices that between (price+percent) and (price-percent), duo the price is a spesific number (10+ digits)
    #Divide those counted prices by the amount of prices.

x_target.mean() # 0.19357003724451638
x_target.mean()/100 # 0.0019357003724451637

maxLim = x_target.mean() + x_target.mean()/100
minLim = x_target.mean() - x_target.mean()/100
counter = x_target.where(x_target > minLim).where(x_target < maxLim).dropna().count()
print (counter / x_target.count())


price    0.015707
dtype: float64


In [381]:
x_target = pd.read_csv(r'C:\Users\avoav\Introduction to Data Science\Project סמסטר א\project-semester-a\TrainSet-target_data.csv' , index_col='index' )
x_target.mean() 


price    104849.31825
dtype: float64

In [382]:
x_target

,price
index,
1608,239000
1329,69900
1782,100000
1193,33000
1166,437320
...,...
1650,75000
1100,29000
1135,110000


In [482]:
#את הדאטה שהקדשנו לטריין אנחנו מפצלים שוב ע"מ לבחון את מודל הרגרסיה הלינארית עם רנדום אחר

x_train1 , x_test1 , y_train1 , y_test1= sklearn.model_selection.train_test_split(x_train, x_target , test_size=0.1 , random_state = 2) 

lin_reg =  LinearRegression()
fit1= lin_reg.fit(x_train1, y_train1)

pred = fit1.predict(x_test1)
#print (f"\n{pred}")
print (f"RMSA: {np.sqrt(mean_squared_error(y_test1 , pred))}\n")

print(lin_reg.score( np.ravel(y_test1, order = 'C'), pred))

RMSA: 82.19160614849096



ValueError: Expected 2D array, got 1D array instead:
array=[438 430 419 801 499 107 555 434 315 209 315 462 688 658 462 479 219 488
 733 252 180 719 419 278 419 326 348 322 548 715 676 387 586 534 482 460
 473 363 708  49 159 376 397 278 485 502  64 397  79 559 323  82 725 499
 355 659   1 479 802 210 215 351  44 415 460 152 219 383 147 273 106 538
 376 417  86 132 368  38 270 747 172 372 499 253 399 261  91 249 234  43
 254 210 197  81 364 300 625 546 191 319 428 568 363 583 615 596  67 400
 118  35 462 231 571 143 824 521  29 518 387 573 365 720  73 204 165 611
 778 705 157 725 800 243  41 486 687 274 125 790 337 677 625 495 462 658
 436 428  23 520 523 589 440 490 115 151 369 512 497  29 565  19 673 622
 822 705 462 597 116 731 595 593 169 265 215 383 276 527 237 770 183 370
 288 680 482 180 495 227  32 571 538 485  68 397 431 338   4 487 242 374
 177 200 666 518 559 489 267 625 729 365 661 694 254 673 165 692 476 344
  46 302 383 476 499 374 603 535 535 315 118 549 540 417 563 590 

In [232]:

#מנסים להבין למה לא הצךלחנו לחזות טוב עם המודל שלנו... עוברים על המכוניות ידנית ורואים באילו מכוניות הטעויות הגדולות קרו
#ננסה לבנות עמודה שהקורלציה שלה עם המחיר תהיה גבוהה
#ננסה לשנות את ה"טראש הולד" של קביעת גודל המכונית ולהכניס מכוניות לקטגוריה "גדול" רק מסף גבוהה יותר

In [479]:
print ("score:" , lin_reg.score(x_test1 , y_test1))

score: -2.049214922384065


In [475]:
param_grid_LR = [  {'C' : [1 ,3 ,5 ] ,  'kernel': ['linear' , 'rbf']} ]

grid_search = GridSearchCV(svm.SVC(gamma = 'auto') , param_grid_LR , cv = 3,
                                   scoring = 'neg_mean_squared_error' , return_train_score = False)
grid_search.fit(x_train1, np.ravel(y_train1, order = 'C'))
df = pd.DataFrame(grid_search.cv_results_)
df[['param_C' , 'param_kernel' , 'mean_test_score']]



,param_C,param_kernel,mean_test_score
0,1,linear,-3.154516e+09
1,1,rbf,-6.357948e+09
2,3,linear,-3.053501e+09
3,3,rbf,-6.031002e+09
4,5,linear,-2.896841e+09
5,5,rbf,-5.918689e+09


In [476]:
print ("best score :" , grid_search.best_score_)
print ("best params:" , grid_search.best_params_)


best score : -2896840794.8349957
best params: {'C': 5, 'kernel': 'linear'}


In [470]:
# השתמשתי ע"מ לסדר את עמודת המחירים של הטריין בפונקציה הזו 
lab_enc = preprocessing.LabelEncoder()
x_target = lab_enc.fit_transform(np.ravel(x_target, order = 'C'))
print(utils.multiclass.type_of_target(x_target))

clf = svm.SVC(kernel='linear', C=1, random_state=2).fit(x_train ,np.ravel(x_target, order = 'C') )
scores = cross_val_score(clf, x_train , x_target, cv=3)
scores


C:\Users\avoav\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


multiclass


C:\Users\avoav\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


array([0.03475336, 0.02693603, 0.03030303])

In [477]:
clf = svm.SVC(kernel='linear', C=5, random_state=2).fit(x_train ,np.ravel(x_target, order = 'C') )
scores = cross_val_score(clf, x_train , x_target, cv=3)
scores

C:\Users\avoav\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


array([0.02690583, 0.03030303, 0.03030303])

In [478]:
# x_target[["price"]] = MinMaxScaler().fit_transform(x_target[["price"]]) 
# x_target = x_target["price"]
def display_scores(scores):
    print ("scores:",  scores)
    print("Mean:" , scores.mean() )
    print ("Standard deviation:" , scores.std()) 
    
    
lin_reg =  LinearRegression()
lin_reg = lin_reg.fit(x_train, x_target)

#חלק מהתוצאות טובות מאוד! 9 , כלומר אחוזי דיוק גבוהים ולכן צריכים לברר איך אנו משיגים את המספר המדוייק שיגרום לתוצאה כזו גם בריצה של הטסט.

scores = cross_val_score(lin_reg, x_train ,x_target ,  scoring = "neg_mean_squared_error", cv=10)
#linReg_rmse_scores = np.sqrt(-scores)

#display_scores(linReg_rmse_scores)
display_scores(scores)
print ("score:" , lin_reg.score(x_test1 , y_test1))

scores: [-6.61573891e+03 -4.87896996e+03 -4.88752783e+03 -5.51820028e+03
 -5.27024251e+03 -5.51429670e+03 -4.60527685e+03 -7.56388600e+03
 -3.21313698e+27 -5.52475954e+26]
Mean: -3.765612929155548e+26
Standard deviation: 9.59765327051744e+26
score: -2.049214922384065


In [327]:
x_train1 , x_test1 , y_train1 , y_test1= sklearn.model_selection.train_test_split(x_train, x_target , test_size=0.1 , random_state = 2) 


tree_reg =  DecisionTreeRegressor()
fit1 = tree_reg.fit(x_train1 , y_train1)
pred = fit1.predict(x_test1)
#print (f"\n{pred}")
print (f"RMSA: {np.sqrt(mean_squared_error(y_test1 , pred))}\n")
print ("score:" , tree_reg.score(x_test1 , y_test1))

RMSA: 74.60243508690264

score: 0.8700715316902132


In [369]:
param_grid_DT = [{'splitter' : ['best' , "random"] , 'max_features':[10, 15 ,20 , 40] , 'min_samples_split': [2 , 3 , 5, 15], }
                ]
grid_search = GridSearchCV(tree_reg , param_grid_DT , cv = 3,
                                   scoring = 'neg_mean_squared_error' , return_train_score = False)
grid_search.fit(x_train1, y_train1)
df = pd.DataFrame(grid_search.cv_results_)

df[['param_max_features' , 'param_min_samples_split' , 'param_splitter' , 'params' , 'mean_test_score']]

print ("best score :" , grid_search.best_score_)
print ("best params:" , grid_search.best_params_)


best score : -5579.592049967673
best params: {'max_features': 40, 'min_samples_split': 5, 'splitter': 'random'}


In [290]:
# שמירה של המודל: 
tree_model =  DecisionTreeRegressor()
tree_model.fit(x_train1 , y_train1)
joblib.dump(tree_model , 'tree-model_1.joblib')

#שימוש במודל קיים : 
model = joblib.load('tree-model_1.joblib')
pred = model.predict(x_test1)
pred

array([ 99000., 108500.,  90000., 445000.,  76000.,  38900., 117000.,
       113900.,  95000.,  52000.,  93000.,  97000., 168000., 420000.,
        99000., 112900.,  39000., 116770., 264355.,  60000.,  53358.,
       245000., 100900.,  53000.,  95000.,  69900., 115000.,  89900.,
       121000., 227000., 285000., 106102., 180000., 108000., 101000.,
       112900.,  92000.,  89000., 420000.,  18000.,  54000.,  87900.,
        76900.,  46000., 116500., 125000.,  55000., 106102.,  36000.,
       168000.,  74000.,  40255., 254900., 113000., 134000., 175900.,
        15900., 112900., 430000.,  50000.,  58000.,  79900.,  23900.,
       115000., 109900.,  35000.,  53900.,  91500.,  41000.,  67000.,
        37460., 109999.,  89000.,  91500.,  32000.,  43450.,  88000.,
        19000.,  68412., 479000.,  49900.,  64000.,  20900.,  75500.,
        82000.,  75900.,  59900.,  59000.,  49900.,  32000.,  73000.,
        51900.,  58000.,  15900.,  79900.,  67000., 168000., 112900.,
        45000.,  899

In [235]:
 # יש שיפור אבל לא מספיק... גם אחרי שהחלטנו לנסות למחוק את התוצאות הקיצוניות מהטבלה.

RMSE: 208.91294681993415


0.8460227694638253

In [ ]:
# אפשר לראות שאם נשתמש ב"מודל אפס" שמחזיר פשוט את הממוצע של המחירים בטבלה
#הטעות חמורה בהרבה מאשר במודל שלנו. 

In [237]:
m = 0
whatCV = 0

for i in np.arange(2, 40):
    ind = cross_val_score(lin_reg, x_train, x_target, cv=i)
    for j in ind:
        if j > m:
            m = j
            whatCV = i
    
print(m)
print("for cv=", i)
# cv=39 זהו ההיפר משתנה הכי טוב שלנו ברגרסיה ליניארית

0.9545137277864899
for cv= 39


In [318]:
# y_test1[["price"]] = MinMaxScaler().fit_transform(y_test1[["price"]])
# y_test1 = y_test1[["price"]]

In [ ]:
#create pipeline:
#pipe = sklearn.pipeline.Pipeline(steps = [('My_model' , ExperimentalTransformer() )])
#pipe.fit(x_train1 , y_train1)
#pred = pipe.transform(y_test1)

# class ExperimentalTransformer(BaseEstimator , TransformerMixin):
#     def _init_ (self , ):
#         print('\n>>>init() called.\n')
#     def fit (self , target , y = None):
#         return self
#     def transform (self , X , y = None):
#         x_ = X.copy()
#         x_.price= x_target.mean()
#         return x_

In [353]:
X_train, X_test, y_train, y_test = train_test_split(x_train,x_target , test_size=0.20, random_state=4)

names = ["Decision Tree", "Linear Regression", "zero model"]

param_grid_DT = [{'splitter' : ['best' , "random"] , 'max_features':[2 , 10 ,20] , 'min_samples_split': [2 , 3 , 5]}
                ]
param_grid_LR =  [  {'C' : [1 ,3 ,5] ,  'kernel': ['linear' , 'rbf']} ]

classifiers = [
    DecisionTreeRegressor()
   , LinearRegression() ]

score = 0
for name, clf in zip(names, classifiers):
    if name == "Decision Tree":
        clf = DecisionTreeRegressor()
        grid_search = GridSearchCV(clf, param_grid=param_grid_DT)
        grid_search.fit(X_train, y_train)
        if grid_search.best_score_ > score:
            score = grid_search.best_score_
            best_clf = grid_search
    
    elif name == "Linear Regression":
        clf = svm.SVC(gamma = 'auto')
        grid_search = GridSearchCV(clf, param_grid_LR , cv = 3,
                                   scoring = 'neg_mean_squared_error' , return_train_score = True)
        grid_search.fit(X_train, y_train)
        if grid_search.best_score_ > score:
            score = grid_search.best_score_
            best_clf = grid_search


print("best classifier: ", best_clf, " Accuracy= ", score)


C:\Users\avoav\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


best classifier:  GridSearchCV(estimator=DecisionTreeRegressor(),
             param_grid=[{'max_features': [2, 10, 20],
                          'min_samples_split': [2, 3, 5],
                          'splitter': ['best', 'random']}])  Accuracy=  0.8460227694638253
